In [ ]:
import numpy as np
from scipy.signal import find_peaks, peak_widths


class PeakFinder:
    """
    Finds and characterises peaks in PL.
    """

    def __init__(self, x_data, y_data):
        """
        Args:
            x_data : array
                Wavelength or energy axis
            y_data : array
                PL intensity
        """
        self.x = np.array(x_data)
        self.y = np.array(y_data)

        self.peaks = None
        self.properties = None
        self.peak_list = []

    
    def find_peaks(self, prominence=0.05, height=None, distance=None):
        """
        Run scipy peak finding
        """

        self.peaks, self.properties = find_peaks(
            self.y,
            prominence=prominence,
            height=height,
            distance=distance
        )

        return self.peaks

    
    def extract_peak_parameters(self):
        """
        Calculate FWHM and output some peak info
        """

        if self.peaks is None:
            raise ValueError("Error: No peaks found")
        
        # FWHM calculation
        widths, width_heights, left_ips, right_ips = peak_widths(
            self.y,
            self.peaks,
            rel_height=0.5
        )

        self.peak_list = []

        for i, peak_index in enumerate(self.peaks):

            # Convert width from index units to x-axis units
            fwhm = (
                self.x[int(right_ips[i])] -
                self.x[int(left_ips[i])]
            )

            peak_dict = {
                "location": self.x[peak_index],
                "intensity": self.y[peak_index],
                "fwhm": fwhm,
                "prominence": self.properties["prominences"][i]
            }

            self.peak_list.append(peak_dict)

        return self.peak_list


In [ ]:
class ZPLClassification:
    """
    Classifies number of Zero-Phonon Lines (ZPLs) in a spectrum.

    Args:
    peaks : list of dict
        Each peak must contain:
        {
            "location": float,
            "fwhm": float,
            "prominence": float
        }

    thresholds : dict
        Threshold values used for classification.
    """

    def __init__(self, peaks, thresholds=None):

        self.peaks = peaks

        # Not sure if this are good parameters
        self.thresholds = thresholds or {
            "max_fwhm": 2.0,           
            "min_prominence": 0.05,   
            "location_range": None    
        }

        self.zpl_peaks = []

    
    def _is_zpl(self, peak):
        """Check if a peak satisfies ZPL conditions"""

        if peak["fwhm"] > self.thresholds["max_fwhm"]:
            return False

        if peak["prominence"] < self.thresholds["min_prominence"]:
            return False

        loc_range = self.thresholds["location_range"]
        if loc_range is not None:
            if not (loc_range[0] <= peak["location"] <= loc_range[1]):
                return False

        return True

    
    def classify(self):
        """Return number of detected ZPLs"""

        self.zpl_peaks = [p for p in self.peaks if self._is_zpl(p)]

        n_zpl = len(self.zpl_peaks)

        if n_zpl == 0:
            return "No ZPL detected"
        elif n_zpl == 1:
            return "Single ZPL"
        elif n_zpl == 2:
            return "Two ZPLs"
        elif n_zpl >= 3:
            return "Multiple ZPLs (3 or more)"



In [1]:
def classify_emitter(peaks, wavelength, zpl_window=(610, 641)):
    '''
    Classifies emitters based on the number of ZPL peaks within a specified wavelength window.

    Args:
        peaks (list): List of peak indices.
        wavelength (list): List of wavelengths corresponding to the peaks.
        zpl_window (tuple): Tuple specifying the wavelength window for ZPL classification.
    Returns:
        str: number of ZPL peaks.
    '''
    
    if len(peaks) == 0:
        return "uncertain"

    peak_wl = wavelength[peaks]
    zpl_peaks = peak_wl[
        (peak_wl > zpl_window[0]) & (peak_wl < zpl_window[1])
    ]

    n = len(zpl_peaks)

    if n == 1:
        return "1_ZPL", zpl_peaks
    elif n == 2:
        return "2_ZPL", zpl_peaks
    elif n >= 3:
        return "3_ZPL", zpl_peaks
    else:
        return "uncertain"
    
